In [1]:
import os
import pandas as pd
from transformers import BertTokenizer, AdamW, BertModel
import torch
from transformers import BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from torch import nn
import time
import datetime
import torch.optim as optim

In [2]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
# Train set
train_pos_folder = './data/aclImdb/train/pos'
train_neg_folder = './data/aclImdb/train/neg'

train_pos_sentences = [open(os.path.join(train_pos_folder, f)).read().strip() for f in os.listdir(train_pos_folder)]
train_neg_sentences = [open(os.path.join(train_neg_folder, f)).read().strip() for f in os.listdir(train_neg_folder)]

train_df = pd.DataFrame({
    'text': train_pos_sentences + train_neg_sentences,
    'label': [1] * len(train_pos_sentences) + [0] * len(train_neg_sentences)  # 1 for positive, 0 for negative
})

In [4]:
# Test set
test_pos_folder = './data/aclImdb/test/pos'
test_neg_folder = './data/aclImdb/test/neg'

test_pos_sentences = [open(os.path.join(test_pos_folder, f)).read().strip() for f in os.listdir(test_pos_folder)]
test_neg_sentences = [open(os.path.join(test_neg_folder, f)).read().strip() for f in os.listdir(test_neg_folder)]

test_df = pd.DataFrame({
    'text': test_pos_sentences + test_neg_sentences,
    'label': [1] * len(test_pos_sentences) + [0] * len(test_neg_sentences)  # 1 for positive, 0 for negative
})

In [5]:
train_df

,text,label
0,Bromwell High is a cartoon comedy. It ran at t...,1
1,Homelessness (or Houselessness as George Carli...,1
2,Brilliant over-acting by Lesley Ann Warren. Be...,1
3,This is easily the most underrated film inn th...,1
4,This is not the typical Mel Brooks film. It wa...,1
...,...,...
24995,"Towards the end of the movie, I felt it was to...",0
24996,This is the kind of movie that my enemies cont...,0
24997,I saw 'Descent' last night at the Stockholm Fi...,0
24998,Some films that you pick up for a pound turn o...,0


In [6]:
# # Create a custom dataset class for our data
# class IMDBDataset(Dataset):
#     def __init__(self, data, tokenizer, max_len):
#         self.data = data
#         self.tokenizer = tokenizer
#         self.max_len = max_len

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         text = self.data.iloc[idx, 0]
#         label = self.data.iloc[idx, 1]

#         encoding = self.tokenizer.encode_plus(
#             text,
#             add_special_tokens=True,
#             max_length=self.max_len,
#             padding='max_length',
#             truncation=True,
#             return_attention_mask=True,
#             return_tensors='pt'
#         )

#         return {
#             'input_ids': encoding['input_ids'].flatten(),
#             'attention_mask': encoding['attention_mask'].flatten(),
#             'labels': torch.tensor(label, dtype=torch.long)
#         }
    
# Create a custom dataset class for our data
class IMDBDataset(Dataset):
    def __init__(self, data, tokenizer, max_len):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx, 0]
        label = self.data.iloc[idx, 1]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.float).unsqueeze(0)
        }

In [7]:
# Load the pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

d:\downloads\downloads_from_browser\anaconda\app\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
# Create data loaders for the training and testing data
train_dataset = IMDBDataset(train_df, tokenizer, max_len=512)
test_dataset = IMDBDataset(test_df, tokenizer, max_len=512)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [9]:
# for batch in train_loader:
#     labels = batch['labels']
#     print(labels.shape)

In [10]:
# Create a custom model for sentiment analysis
class SentimentAnalysisModel(nn.Module):
    def __init__(self, bert_model):
        super(SentimentAnalysisModel, self).__init__()
        self.bert_model = bert_model
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.bert_model.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert_model(input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        outputs = self.classifier(pooled_output)
        return outputs

In [11]:
model = BertModel.from_pretrained('bert-base-uncased')
model = SentimentAnalysisModel(model)
model.to(device)
print(model)

SentimentAnalysisModel(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-

In [12]:
input_ids = torch.randint(0, 100, (32, 512)).to(device)  # random input IDs
attention_mask = torch.ones((32, 512)).to(device)  # random attention mask

output = model(input_ids, attention_mask)
print(output.shape)

d:\downloads\downloads_from_browser\anaconda\app\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


torch.Size([32, 1])


In [13]:
# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-5)

# Define the loss function
# criterion = nn.BCELoss()
criterion = nn.BCEWithLogitsLoss()

In [14]:
# # Train the model
# for epoch in range(3):
#     model.train()
#     total_loss = 0
#     for batch in train_loader:
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)

#         optimizer.zero_grad()

#         outputs = model(input_ids, attention_mask)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         print(total_loss)
#         total_loss += loss.item()
#     print(f'Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}')

In [15]:

# Train the model
for epoch in range(3):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}')


KeyboardInterrupt: 